In [636]:
import pandas as pd
import ast

ads = pd.read_excel('ad_df_gb.xlsx')
mod = pd.read_excel('mod_df_lr.xlsx')
mod["market"] = mod["queue_market"].apply(lambda x: ast.literal_eval(x))

mod.columns


Index(['Productivity', 'Utilisation %', 'handling time', 'accuracy',
       'moderator', 'queue_market', 'mod_score_pred', 'market'],
      dtype='object')

#### We create an efficiency column that is a function of the three factors that counts towards a moderator's efficiency level. Productivity , Utilisation and Handling Time. 

#### We have tried various combinations of weights for productivity and utilisation and have come out with the ration of 0.4:0.6 ratio. The primary goal of this task is to focus on resource optimization and cost control, thus justifiying the prioritization of utilisation and assigning it a higher weight. 


In [637]:
import math

# check for prod / utilisation = 0
rows_with_utilisation_zero = mod[mod['Utilisation %'] == 0]
rows_with_utilisation_zero['Utilisation %'] = 0.00001
mod.update(rows_with_utilisation_zero)
rows_with_prod_zero = mod[mod['Productivity'] == 0]
rows_with_prod_zero["Productivity"] = 0.00001
mod.update(rows_with_prod_zero)
rows_with_hand_zero = mod[mod['handling time'] == 0]
rows_with_hand_zero["handling time"] = 0.00001
mod.update(rows_with_hand_zero)

mod['efficiency'] = (0.5 * mod['Productivity'] + 0.5 * mod['Utilisation %']).astype(float) / (mod['handling time'] ** 2)
mod.head()

/var/folders/1t/nll74_0j61918pgb1cz1kwsr0000gn/T/ipykernel_22591/2041000100.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_utilisation_zero['Utilisation %'] = 0.00001
/var/folders/1t/nll74_0j61918pgb1cz1kwsr0000gn/T/ipykernel_22591/2041000100.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_prod_zero["Productivity"] = 0.00001
/var/folders/1t/nll74_0j61918pgb1cz1kwsr0000gn/T/ipykernel_22591/2041000100.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,Productivity,Utilisation %,handling time,accuracy,moderator,queue_market,mod_score_pred,market,efficiency
0,6.206456,4.422678,2.523789,4.738351,6.073125e+06,"[""JP""]",3.797080,[JP],0.834376
1,5.289222,4.369743,1.184752,5.218638,1.743844e+15,"[""JP""]",3.552170,[JP],3.440694
2,4.928172,1.821504,2.965263,4.372760,1.912879e+06,"[""TW""]",3.484792,[TW],0.383819
3,4.775042,4.608005,2.662661,5.075269,1.714052e+15,"[""FR""]",3.503785,[FR],0.661732
4,4.729308,4.390590,2.386357,4.329749,1.737386e+15,"[""MY""]",3.624446,[MY],0.800735


In [638]:
mod["efficiency"].describe()

inf_rows = mod[mod['efficiency'] == mod["efficiency"].max()]
inf_rows

,Productivity,Utilisation %,handling time,accuracy,moderator,queue_market,mod_score_pred,market,efficiency
1273,0.003237,0.00001,0.00001,3.053763,1598527.0,"[""US"", ""CA""]",2.799196,"[US, CA]",1.623690e+07


In [639]:

country_moderator_dict = {}
for index, row in mod.iterrows():
    countries = row['market']
    moderator_id = row['moderator']
    y_pred = row['mod_score_pred']
    efficiency = row["efficiency"]

    for country in countries:
        # Check if the country is already in the dictionary
        if country in country_moderator_dict:
            country_moderator_dict[country].append((moderator_id,y_pred,efficiency))
        else:
            # If the country is not in the dictionary, create a new entry with a list containing the moderator_id
            country_moderator_dict[country] = [(moderator_id,y_pred,efficiency)]


In [640]:
country_moderator_dict

{'JP': [(6073125.0, 3.797079797089214, 0.834376254722555),
  (1743843711746081.0, 3.552170223621821, 3.4406939695095504),
  (5338213.0, 3.562036316979611, 0.9262668669615809),
  (8585325.0, 3.606896662418727, 1.0720584829106679),
  (2613990.0, 3.458312168289836, 2.3829517318037543),
  (2712226.0, 3.438833272394122, 0.6405362662656818),
  (8959882.0, 3.301787691189214, 0.9781218491410093),
  (1669008361646085.0, 3.228263165541589, 0.3644333437932705),
  (9211081.0, 3.211964941523489, 1.3398393223249867),
  (1972062.0, 3.206602713601618, 0.3688247521724568),
  (1680306084527105.0, 3.094620384945041, 1.3691621022915503),
  (7977305.0, 3.132716210957223, 0.6027280676199603),
  (1771766145225762.0, 2.826783944488553, 2.830545308159737),
  (1649165526729733.0, 3.001126153777073, 1.0982002077221402),
  (1718358.0, 3.169428097947722, 0.4007431284627897),
  (1658495581892613.0, 2.969471801809215, 0.5552419877019119),
  (1735754088302593.0, 2.82633993943411, 0.30176933243788),
  (177045760073625

In [641]:
mod_sorted_dict = {
    country: sorted(ad_data, key=lambda x: (x[1], x[2]), reverse=True)
    for country, ad_data in country_moderator_dict.items()
}
mod_sorted_dict

{'JP': [(6073125.0, 3.797079797089214, 0.834376254722555),
  (8585325.0, 3.606896662418727, 1.0720584829106679),
  (5338213.0, 3.562036316979611, 0.9262668669615809),
  (1743843711746081.0, 3.552170223621821, 3.4406939695095504),
  (2613990.0, 3.458312168289836, 2.3829517318037543),
  (2712226.0, 3.438833272394122, 0.6405362662656818),
  (8959882.0, 3.301787691189214, 0.9781218491410093),
  (1669008361646085.0, 3.228263165541589, 0.3644333437932705),
  (9211081.0, 3.211964941523489, 1.3398393223249867),
  (1972062.0, 3.206602713601618, 0.3688247521724568),
  (1718358.0, 3.169428097947722, 0.4007431284627897),
  (7977305.0, 3.132716210957223, 0.6027280676199603),
  (2570177.0, 3.095797226626654, 0.04429273683411469),
  (1680306084527105.0, 3.094620384945041, 1.3691621022915503),
  (1649165526729733.0, 3.001126153777073, 1.0982002077221402),
  (6953937.0, 2.981984634591248, 0.002212908365507878),
  (1658495581892613.0, 2.969471801809215, 0.5552419877019119),
  (2972398.0, 2.9592252755580

In [642]:
ads_dict = {}
for index, row in ads.iterrows():
    ad_id = row['ad_id']
    y_pred = row['ad_score_pred']
    country =  row['delivery_country']
    if country in ads_dict:
        ads_dict[country].append((ad_id,y_pred))
    else:
        ads_dict[country] = [(ad_id,y_pred)]

ads_sorted_dict = {
    country: sorted(ad_data, key=lambda x: x[1], reverse=True)
    for country, ad_data in ads_dict.items()
}


In [643]:
print(ads_sorted_dict)


{'US': [(1770790523232290, 44.64640508394639), (1773325462877200, 44.51817600202583), (1772501230876700, 44.49365800774512), (1773054455743500, 44.32404752800682), (1773055190177790, 44.29765620444273), (1773331161559080, 44.25814498423188), (1773584657728530, 44.24765974180046), (1773585343379450, 44.22328253000613), (1773414483579950, 44.218196343804), (1773414483579950, 44.21514778865794), (1773054627210240, 44.21059675506937), (1773142749094960, 44.1468714482341), (1772968751779870, 44.1374394314587), (1772360893228080, 44.09079661652097), (1768629614968860, 44.05943252989478), (1773051963819040, 44.0590387867979), (1772422251351050, 44.05263018557855), (1771871833741360, 44.03517518786425), (1773595150429240, 44.02757136941227), (1770164604637180, 44.02703547126158), (1771882804124670, 44.02182075986875), (1769869625886720, 44.02023507965569), (1772331510445100, 44.01813414663091), (1773587516600320, 44.00041049553636), (1773465813379070, 43.94535885519791), (1771977492942860, 43.

In [644]:
## get count of ads in mods _ dict 
country_counts_mods = {}

for country, mods in mod_sorted_dict.items():
    count = len(mods)
    country_counts_mods[country] = count

for country, count in country_counts_mods.items():
    print(f"{country}: {count}")

JP: 52
TW: 30
FR: 33
MY: 59
KR: 34
UA: 12
RU: 13
GB: 130
IE: 96
US: 143
CA: 135
SA: 61
OM: 61
BH: 59
AE: 19
KW: 64
MA: 36
EG: 64
LB: 56
NL: 22
MX: 110
UY: 87
CO: 107
CL: 108
AR: 109
EC: 88
PE: 108
BR: 109
PK: 27
LT: 2
HR: 2
ZA: 17
QA: 63
TR: 28
LV: 3
ME: 4
AU: 27
NZ: 22
SG: 34
BE: 2
FI: 13
CZ: 14
PL: 21
JO: 58
IQ: 53
RO: 19
PH: 57
TH: 60
SE: 16
HU: 16
IT: 20
ID: 109
KH: 17
VN: 116
SI: 1
PT: 16
ES: 22
DK: 12
DZ: 12
TN: 12
NO: 12
GR: 17
OTHER: 3
BY: 9
DE: 40
KZ: 15
AT: 5
CH: 5
IL: 9
UZ: 1
BG: 1
CY: 1


In [645]:
## add to data frame 
matching_df = pd.DataFrame(list(country_counts_mods.items()), columns=['Country', 'Num_Moderators'])
matching_df

,Country,Num_Moderators
0,JP,52
1,TW,30
2,FR,33
3,MY,59
4,KR,34
...,...,...
67,CH,5
68,IL,9
69,UZ,1
70,BG,1


In [646]:
## get count of ads in mods _ dict 
country_counts_ads = {}

for country, mods in ads_sorted_dict.items():
    count = len(mods)
    country_counts_ads[country] = count

for country, count in country_counts_ads.items():
    print(f"{country}: {count}")

US: 3095
SA: 2722
AU: 460
NZ: 232
FI: 164
JP: 522
MY: 1270
TH: 2549
ID: 2657
TR: 944
VN: 2799
BR: 1997
HU: 350
KW: 361
ZA: 134
DE: 541
CA: 631
GB: 954
MA: 120
AR: 43
BY: 21
CL: 68
NL: 213
EC: 65
GR: 421
UY: 43
PL: 414
DK: 192
CH: 125
AT: 114
IL: 390
PE: 120
CO: 83
RO: 347
MX: 204
IE: 254
PT: 170
CZ: 150
BE: 173
SE: 205
SG: 214
PH: 907
AE: 509
KR: 757
IT: 493
EG: 353
KH: 370
ES: 378
BH: 43
TW: 334
NO: 248
PK: 272
FR: 548
LB: 30
QA: 189
OM: 32
JO: 32
IQ: 190
KZ: 137


In [647]:
## create df to show counts of advertisments 
ads_data = pd.DataFrame(list(country_counts_ads.items()), columns=["Country", "Num_Ads"])
ads_data

,Country,Num_Ads
0,US,3095
1,SA,2722
2,AU,460
3,NZ,232
4,FI,164
5,JP,522
6,MY,1270
7,TH,2549
8,ID,2657
9,TR,944


In [648]:
# get mean baseline_st
describe_baseline = ads_sorted["baseline_st"].describe()
median_complexity = ads_sorted["baseline_st"].median()
describe_baseline


count    32353.000000
mean         0.629129
std          0.516872
min          0.000000
25%          0.000000
50%          0.798387
75%          1.000000
max          5.685484
Name: baseline_st, dtype: float64

In [649]:
ads_sorted

,ad_id,ad_score_pred,delivery_country,baseline_st
152,1769826449034250,44.758500,CA,1.024194
1871,1770790523232290,44.646405,US,1.024194
15125,1773325462877200,44.518176,US,0.806452
2145,1769733020810290,44.498459,JP,5.685484
2137,1772501230876700,44.493658,US,1.024194
...,...,...,...,...
487,1773428471490560,10.220458,VN,0.000000
492,1773429522808830,10.197187,VN,0.000000
491,1773429428414460,10.197187,VN,0.000000
488,1773428783819770,10.172745,VN,0.000000


#### We strongly feel that an advertisement from Country A should be reviewed by a moderator from Country A. Thus, we have decided to group the matching algorithm by countries. For each country, we look into the number of ads and the number of moderators who are able to review said country. We then calculate the mean_ads per moderators through ads / moderators. This gives us the very preliminary mean number of ads that is to be reviewed by each moderator. However, we also need to take into consideration the 

In [650]:
merged_df = pd.merge(ads_data, matching_df, on="Country", how="outer").fillna(0)
merged_df["Mean_Ads"] = merged_df["Num_Ads"] / merged_df["Num_Moderators"]

merged_df["Mean_Ads"] = merged_df["Mean_Ads"].apply(lambda x: max(1, x))

# # total copmlexity score per peron = mean ads * mean baseline_st
merged_df["total_complexity_score"] = merged_df["Mean_Ads"] * median_complexity
merged_df

,Country,Num_Ads,Num_Moderators,Mean_Ads,total_complexity_score
0,US,3095.0,143,21.643357,17.279777
1,SA,2722.0,61,44.622951,35.626388
2,AU,460.0,27,17.037037,13.602151
3,NZ,232.0,22,10.545455,8.419355
4,FI,164.0,13,12.615385,10.071960
...,...,...,...,...,...
67,TN,0.0,12,1.000000,0.798387
68,OTHER,0.0,3,1.000000,0.798387
69,UZ,0.0,1,1.000000,0.798387
70,BG,0.0,1,1.000000,0.798387


In [651]:
## add to dictionary 
country_mean_ads_df = pd.DataFrame(merged_df)

# Create a dictionary from the DataFrame
country_mean_ads_dict = country_mean_ads_df.set_index('Country')['total_complexity_score'].to_dict()


sorted_country_total_score = dict(sorted(country_mean_ads_dict.items(), key=lambda item: item[1], reverse=True))
sorted_country_total_score

{'BE': 69.06048387096774,
 'SA': 35.62638815441565,
 'IL': 34.596774193548384,
 'TH': 33.91814516129032,
 'TR': 26.91705069124424,
 'AE': 21.3883701188455,
 'CH': 19.959677419354836,
 'GR': 19.771821631878556,
 'IT': 19.68024193548387,
 'ID': 19.461601065403965,
 'VN': 19.264530033370413,
 'AT': 18.203225806451613,
 'KR': 17.775853889943075,
 'HU': 17.464717741935484,
 'KH': 17.376660341555976,
 'US': 17.279776674937963,
 'MY': 17.185620557681794,
 'NO': 16.5,
 'PL': 15.73963133640553,
 'BR': 14.627330571174902,
 'RO': 14.58106960950764,
 'ES': 13.717741935483872,
 'AU': 13.602150537634408,
 'FR': 13.25806451612903,
 'DK': 12.774193548387096,
 'PH': 12.70415959252971,
 'DE': 10.798185483870968,
 'SE': 10.229334677419354,
 'FI': 10.071960297766749,
 'TW': 8.888709677419355,
 'CZ': 8.554147465437786,
 'PT': 8.482862903225806,
 'NZ': 8.419354838709676,
 'PK': 8.043010752688172,
 'JP': 8.014578163771711,
 'NL': 7.729838709677419,
 'KZ': 7.291935483870967,
 'ZA': 6.293168880455408,
 'GB': 5

In [652]:
columns_to_select = ["ad_id", "ad_score_pred", "delivery_country","baseline_st"]
ads_df2 = pd.read_excel("ad_df_gb.xlsx", usecols=columns_to_select)
ads_sorted = ads_df2.sort_values(by='ad_score_pred', ascending=False)
# ads_sorted.to_excel('ad_sorted.xlsx', index=False)


In [653]:

mod_counter = {}
data = []
df = pd.DataFrame(columns=["ad_id", "mod_id", "country"])

for index, row in ads_sorted.iterrows():
    country = row["delivery_country"]
    ad_id = row["ad_id"]
    complex_score = row['baseline_st']
    mod_list = mod_sorted_dict[country]
    # get a moderator from this country
    max_complexity = sorted_country_total_score[country]
    for mod_id, y_value, efficiency in mod_list:
        if mod_id not in mod_counter:
            # get the efficiency of the moderator 
            mod_counter[mod_id] = 0.5 * complex_score / efficiency ** 2
            data.append([ad_id,mod_id,country])
            # print(f'{mod_id} : {ad_id}')
            break
        else:
            if mod_counter[mod_id] + 0.5 *  complex_score / efficiency  ** 2 <= max_complexity:
                mod_counter[mod_id] += 0.5 * complex_score / efficiency ** 2
                # print(f'{mod_id} : {ad_id}')
                data.append([ad_id, mod_id, country])
                break
            else:
                continue


df_matched = pd.DataFrame(data, columns=["ad_id", "mod_id", "country"])
df_matched.to_excel('lol.xlsx', index = None)
# df_matched.head()

In [654]:
num_mods = df_matched['mod_id'].unique()
len(num_mods)


534

In [655]:
## what was not matched
not_matched = []
for country in mod["moderator"]:
    if country not in num_mods:
        not_matched.append(country)
not_matched

[9152229.0,
 6715607.0,
 1527988.0,
 1960716.0,
 1672279992480774.0,
 1738944455652353.0,
 8750521.0,
 1705052601354242.0,
 2927720.0,
 1672026730048518.0,
 8920962.0,
 1722268364673026.0,
 6227566.0,
 9672999.0,
 1772379228620801.0,
 7752505.0,
 1726740282128385.0,
 1737385910931458.0,
 1730409705792513.0,
 6217365.0,
 9082283.0,
 2306691.0,
 5563535.0,
 1695027942470657.0,
 2779680.0,
 5569930.0,
 1735529545413634.0,
 2322011.0,
 2062837.0,
 8636306.0,
 1704414376583169.0,
 6085577.0,
 1691852985713666.0,
 1736753775507458.0,
 9075895.0,
 1691841339811842.0,
 9656891.0,
 6517798.0,
 1701877328285698.0,
 1721013868565506.0,
 1686669003006977.0,
 9221676.0,
 1703429274086465.0,
 2855138.0,
 2713299.0,
 1736487777311745.0,
 1193822.0,
 1301929.0,
 3139936.0,
 7550336.0,
 1268890.0,
 7568002.0,
 1749551020194849.0,
 3376191.0,
 8886189.0,
 6251180.0,
 5608898.0,
 1731051715822594.0,
 9256168.0,
 2658129.0,
 6353600.0,
 1695538424423426.0,
 6997655.0,
 5311062.0,
 9265717.0,
 170459656966

### Mean productivity of those unselected moderators is 1.44 (scaled) which is lesser than that of the mean of the entire dataset which is 1.72

In [656]:
filtered_mod = mod[mod['moderator'].isin(not_matched)]
filtered_mod["Productivity"].describe()

count    741.000000
mean       1.546433
std        0.699765
min        0.000010
25%        1.072453
50%        1.513463
75%        1.951413
max        3.946683
Name: Productivity, dtype: float64

In [657]:
mod["Productivity"].describe()

count    1275.000000
mean        1.726160
std         0.834403
min         0.000010
25%         1.166863
50%         1.614808
75%         2.166863
max         6.206456
Name: Productivity, dtype: float64

### Similarly, the mean Utilisation % (scaled) is also lower than that of the mean of the entire dataset

In [658]:
filtered_mod = mod[mod['moderator'].isin(not_matched)]
filtered_mod["Utilisation %"].describe()

count    741.000000
mean       3.702615
std        1.082748
min        0.000010
25%        3.335004
50%        4.067196
75%        4.438750
max        5.844082
Name: Utilisation %, dtype: float64

In [659]:
mod["Utilisation %"].describe()

count    1275.000000
mean        3.798500
std         1.023998
min         0.000010
25%         3.451038
50%         4.085860
75%         4.451038
max         6.743789
Name: Utilisation %, dtype: float64

### Whereas for the handling time, the lower the handling time, the better. Those unselected moderators have a higher mean time of 2.92 compared to the mean of the entire dataset at 2.74. 

In [660]:
filtered_mod = mod[mod['moderator'].isin(not_matched)]
filtered_mod["handling time"].describe()

count    741.000000
mean       2.854004
std       12.276849
min        0.000010
25%        1.354064
50%        2.064354
75%        2.554229
max      242.995857
Name: handling time, dtype: float64

In [661]:
mod["handling time"].describe()

count    1275.000000
mean        2.748678
std         9.643279
min         0.000010
25%         1.613038
50%         2.207627
75%         2.613038
max       242.995857
Name: handling time, dtype: float64

### In conclusion, though there could be other means to utilise these moderators to review ads that are of less importance.  470/1275 moderators which are unused should undergo checks to review their effiency. Their effieciency levels should be raised, else the hiring and utilisation of these moderators do not prove to be profit-maximising for TikTok. Whereas for the ads 30k / 32k were matched to a moderator. Similarly, while there are indeed possible means to find ways to match these unmatched ads, the extremey low priority scores of these ads permits the non-reviews of these ads.


In [662]:
mod_counter

{3371590.0: 16.984066378994438,
 6073125.0: 7.749622726570703,
 1762949587637281.0: 14.455322192413897,
 8716810.0: 5.5488158486483465,
 1714052438738946.0: 13.112761294315895,
 3092815.0: 10.661879330053972,
 3502216.0: 16.249198501803072,
 9020538.0: 34.18829089905416,
 1251392.0: 16.041278904640897,
 7559160.0: 14.082055057141506,
 3725295.0: 17.137330340127225,
 1772483946030082.0: 13.237188418141816,
 1769302206223394.0: 7.299355232188962,
 6525737.0: 32.669650580987444,
 7960156.0: 17.667495744309186,
 1715218806419458.0: 7.092089143618077,
 9060023.0: 3.57461251487569e-08,
 1702615776441346.0: 8.787724265362591,
 3215286.0: 0.00484508361366945,
 8003837.0: 4.488420279306606,
 5090932.0: 13.135402652652234,
 1958955.0: 35.39329660979659,
 6587921.0: 14.31547401160287,
 1752804614390817.0: 14.53689729124881,
 8585325.0: 17.54910818662038,
 6738735.0: 32.43506010598541,
 9630899.0: 14.149696514465209,
 1749551102734338.0: 32.99987503253859,
 9179505.0: 13.072354936550559,
 17520906

In [663]:
distinct_countries = df_matched['country'].unique()


matched_countries = list(distinct_countries)
matched_countries

['CA',
 'US',
 'JP',
 'BR',
 'GB',
 'FR',
 'DE',
 'PL',
 'NO',
 'ES',
 'NL',
 'TH',
 'AU',
 'KZ',
 'IT',
 'TW',
 'CH',
 'SG',
 'AT',
 'NZ',
 'LB',
 'CO',
 'MX',
 'KR',
 'TR',
 'RO',
 'DK',
 'SA',
 'IL',
 'KW',
 'GR',
 'SE',
 'PH',
 'CZ',
 'HU',
 'FI',
 'PT',
 'IE',
 'VN',
 'MA',
 'MY',
 'ID',
 'AE',
 'EC',
 'BE',
 'AR',
 'QA',
 'CL',
 'PE',
 'ZA',
 'KH',
 'UY',
 'PK',
 'EG',
 'IQ',
 'OM',
 'JO',
 'BH',
 'BY']

In [664]:
keys_list = list(sorted_country_total_score.keys())
keys_list
len(keys_list)

72

In [665]:
## what was not matched
not_matched = []
for country in keys_list:
    if country not in matched_countries:
        not_matched.append(country)
not_matched

['UA',
 'RU',
 'LT',
 'HR',
 'LV',
 'ME',
 'SI',
 'DZ',
 'TN',
 'OTHER',
 'UZ',
 'BG',
 'CY']

In [666]:
# Filter the DataFrame based on the 'Country' column
filtered_df = merged_df[merged_df['Country'].isin(not_matched)]
len(filtered_df)

filtered_df

,Country,Num_Ads,Num_Moderators,Mean_Ads,total_complexity_score
59,UA,0.0,12,1.0,0.798387
60,RU,0.0,13,1.0,0.798387
61,LT,0.0,2,1.0,0.798387
62,HR,0.0,2,1.0,0.798387
63,LV,0.0,3,1.0,0.798387
64,ME,0.0,4,1.0,0.798387
65,SI,0.0,1,1.0,0.798387
66,DZ,0.0,12,1.0,0.798387
67,TN,0.0,12,1.0,0.798387
68,OTHER,0.0,3,1.0,0.798387
